In [2]:
import pandas as pd

In [12]:
data = pd.read_csv('./朴素贝叶斯处理之前的全部数据.csv')
data = data.rename(columns={'x': '相关性', 'X':'涨跌'})
data

,publish_time,view_counts,summarization,相关性,涨跌
0,2010-07-21 16:58:34,1244,金价格在亚洲盘时段上行趋势线后遭遇强力阻力收出中等长度的下影线看涨的因素还在待到债务危机避险...,1,1.0
1,2010-07-30 11:47:39,1863,从日K线来看黄金处于交易日大幅走低跌破5日均线后大幅回落至200日平均线即115附近获得支撑,1,1.0
2,2010-08-04 07:48:00,7636,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...,0,0.0
3,2010-08-18 02:02:55,855,捕鱼收网现货金多头在1230美元上方获利平仓美国制造业数据降避险情绪继续升但是美元与黄金分道...,1,0.0
4,2010-08-19 09:49:28,2413,这里是不是做黄金的比较多小弟有意向往这方面发展可惜不太了解希望老手们有空多多指教,1,1.0
...,...,...,...,...,...
113111,2024-08-09 23:39:20,101,白银暴跌黄金硬撑白银反弹黄金暴涨资本根据各自散户的持仓基数分别收割,N,2.0
113112,2024-08-10 00:42:22,85,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口,N,2.0
113113,2024-08-10 02:23:27,125,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...,N,2.0
113114,2024-08-10 15:14:49,313,金价贵吗全球黄金市值16万亿美元加密货币市值都2 6万亿美元虚拟的不到10年就这么高市值黄金...,N,2.0


In [14]:
import pandas as pd
import re
import jieba
#构建中文分词类v3.0
class Leijieba:    
    
    __stopwords = ''
        
    def __init__(self,s,t='',b=''):    #t为停用词的文件参数（路径+文件名）；b为本地词库的文件参数（路径+文件名）
        self.__s = s
        
        if t != '':
            self.__stopwords = [line.strip() for line in open(t,encoding='UTF-8').readlines()]  #逐行读取停用词文件，放入类内部变量__stopwords
            
        if b != '':
            jieba.load_userdict(b)
        
    def cut(self):
        seg_list = jieba.cut(self.__s, cut_all=False)
        if self.__stopwords != '':
            outstr = []    #目的是存储去掉停用用词的分词结果
            for word in list(seg_list):
                if word not in self.__stopwords:
                    if word.strip() !='':   #去空格
                        outstr.append(word)
                        
            seg_list = outstr                                                
        return list(seg_list)
    
    def num(self):
        lst = self.cut()
        
        word_dict = {}        
        for item in lst:
            if item not in word_dict:
                word_dict[item] = 1   #新建“键-值”
            else:
                word_dict[item] += 1  #增加 键的值
        return sorted(word_dict.items(), key=lambda e:e[1], reverse=True)  
    
    def cut1(self,num = 0,sw = 0):   #1.删除词频低于lnum的词,2.if sw ==1,删除单字的词
        lst = self.cut()
        
        word_dict = {}        
        for item in lst:
            if item not in word_dict:
                word_dict[item] = 1   #新建“键-值”
            else:
                word_dict[item] += 1  #增加 键的值
        dic = dict(sorted(word_dict.items(), key=lambda e:e[1], reverse=True)) 
        lst1 = lst[:]
        
        if sw == 1:
            for item in lst:
                if len(item) == 1:
                    lst1.remove(item)
                else:
                    if dic[item] <= num:
                        lst1.remove(item)
        else:
            for item in lst:
                if dic[item] <= num:
                    lst1.remove(item)
                
        return lst1

In [16]:
s = 'AAA我是分词的间隔符BBB'.join(list(data['summarization']))

In [18]:
m = Leijieba(s,t='./stopwords.txt',\
             b='./userdic.txt')
s2 = ' '.join(m.cut1(10,1))
txtf_news = open('./cnews.jieba1.txt','wt', encoding='utf-8')
txtf_news.write(s2.replace('AAA我是分词的间隔符BBB','\n'))
txtf_news.close()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86155\AppData\Local\Temp\jieba.cache
Loading model cost 0.632 seconds.
Prefix dict has been built successfully.


In [22]:
txt = pd.read_csv('./cnews.jieba1.txt',\
                  skip_blank_lines=False,encoding = 'utf-8', header = None)
outer_join = data.join(txt, how='outer')
outer_join.columns = ['publish_time','view_counts','summarization','相关性','涨跌','x']

In [58]:
outer_join

,publish_time,view_counts,summarization,相关性,涨跌,x
0,2010-07-21 16:58:34,1244,金价格在亚洲盘时段上行趋势线后遭遇强力阻力收出中等长度的下影线看涨的因素还在待到债务危机避险...,1,1.0,价格 亚洲 时段 上行 趋势 线后 遭遇 强力 阻力 中等 下影线 看涨 因素 债务 危机 ...
1,2010-07-30 11:47:39,1863,从日K线来看黄金处于交易日大幅走低跌破5日均线后大幅回落至200日平均线即115附近获得支撑,1,1.0,黄金 处于 交易日 大幅 走低 跌破 均线 大幅 回落 200 平均线 115 支撑
2,2010-08-04 07:48:00,7636,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...,0,0.0,黄金 每克 价格 太贵 没人 房子 工资 十年 1000 一月 石油 120 爆跌 可怕 ...
3,2010-08-18 02:02:55,855,捕鱼收网现货金多头在1230美元上方获利平仓美国制造业数据降避险情绪继续升但是美元与黄金分道...,1,0.0,现货 多头 1230 美元 上方 获利 平仓 美国 制造业 数据 避险 情绪 美元 黄金 ...
4,2010-08-19 09:49:28,2413,这里是不是做黄金的比较多小弟有意向往这方面发展可惜不太了解希望老手们有空多多指教,1,1.0,黄金 有意 发展 可惜 不太 希望 老手 有空
...,...,...,...,...,...,...
113111,2024-08-09 23:39:20,101,白银暴跌黄金硬撑白银反弹黄金暴涨资本根据各自散户的持仓基数分别收割,N,2.0,白银 暴跌 黄金 白银 反弹 黄金 暴涨 资本 散户 持仓 收割
113112,2024-08-10 00:42:22,85,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口,N,2.0,黄金 只能 下跌 黄金 赚钱
113113,2024-08-10 02:23:27,125,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...,N,2.0,原油 外汇 黄金 国际 贵金属
113114,2024-08-10 15:14:49,313,金价贵吗全球黄金市值16万亿美元加密货币市值都2 6万亿美元虚拟的不到10年就这么高市值黄金...,N,2.0,金价 全球 黄金 市值 万亿美元 货币 市值 万亿美元 虚拟 不到 市值 黄金


In [60]:
outer_join.to_csv('./分词过的全部数据.csv',index = False)

#######

#####

In [124]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', max_features=5000,  lowercase = False)
vectorizer.fit(list(outer_join['x']))
X = vectorizer.transform(list(outer_join['x']))

In [126]:
X.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.25837178, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [128]:
outer_join1 = outer_join.iloc[:2035]
print(outer_join1['相关性'].unique())

['1' '0']


In [130]:
import numpy as np
ordering = [0,1]
y = np.array(outer_join1['相关性'].map(lambda x: ordering.index(int(x))))

In [132]:
X

<113116x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 1442029 stored elements in Compressed Sparse Row format>

In [134]:
X_2035 = X[:2035]

In [136]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_2035, y, random_state=1)

In [138]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [140]:
y_pred = classifier.predict(X_test)

In [142]:
from sklearn.metrics import classification_report
target_names = [str(i) for i in ordering]
t = classification_report(y_test, y_pred, target_names=target_names)
print(t)

              precision    recall  f1-score   support

           0       0.86      0.21      0.34       112
           1       0.82      0.99      0.90       397

    accuracy                           0.82       509
   macro avg       0.84      0.60      0.62       509
weighted avg       0.83      0.82      0.77       509



In [146]:
outer_join2 = outer_join.iloc[2035:]
outer_join2 = outer_join2.reset_index(drop=True)
outer_join2

,publish_time,view_counts,summarization,相关性,涨跌,x
0,2011-09-24 13:47:40,2409,受股市暴跌拖累隔夜国际金价也出现暴下跌最低下跌至1720美元,N,0.0,股市 暴跌 拖累 国际 金价 下跌 最低 下跌 1720 美元
1,2011-09-24 18:09:09,1.2万,思维转换之快终于在黄金身上体现的淋漓尽至上2000的梦还没做醒马上又要被熊来了的观点占拒淡定...,N,1.0,思维 转换 终于 黄金 体现 至上 2000 观点 轻易 判断 交易日 市场 方向 黄金 上涨
2,2011-09-24 21:18:04,867,美国经济周期研究所周五(9月16日)称美国上调黄金白银期货交易日的领先指标小幅回落至122 ...,N,0.0,美国 研究所 周五 美国 上调 黄金 白银 期货交易 领先 指标 小幅 回落 122 此前...
3,2011-09-24 21:18:58,728,9月23日纽约商品交易所黄金期货价格12月合约周五每盎司暴跌101,N,0.0,纽约商品交易所 黄金 期货价格 合约 周五 每盎司 暴跌 101
4,2011-09-25 09:48:30,2490,话虽偏颇了些但比之股市和大宗商品动者几倍甚致十几倍的涨幅相比黄金明显是落后了我们只是从去年才...,N,1.0,股市 大宗 商品 几倍 涨幅 相比 黄金 落后 去年 大规模 购买 黄金 警惕 眼光
...,...,...,...,...,...,...
111076,2024-08-09 23:39:20,101,白银暴跌黄金硬撑白银反弹黄金暴涨资本根据各自散户的持仓基数分别收割,N,2.0,白银 暴跌 黄金 白银 反弹 黄金 暴涨 资本 散户 持仓 收割
111077,2024-08-10 00:42:22,85,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口,N,2.0,黄金 只能 下跌 黄金 赚钱
111078,2024-08-10 02:23:27,125,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...,N,2.0,原油 外汇 黄金 国际 贵金属
111079,2024-08-10 15:14:49,313,金价贵吗全球黄金市值16万亿美元加密货币市值都2 6万亿美元虚拟的不到10年就这么高市值黄金...,N,2.0,金价 全球 黄金 市值 万亿美元 货币 市值 万亿美元 虚拟 不到 市值 黄金


In [148]:
X_1 = vectorizer.transform(list(outer_join2['x']))

In [152]:
y_pred_1 = classifier.predict(X_1)
print(y_pred_1)

[1 1 1 ... 1 1 1]


In [154]:
for i in range(0,111081):
    outer_join2.loc[i,'相关性'] = y_pred_1[i]

In [160]:
outer_join2 = outer_join2.loc[outer_join2['相关性'] != 0]

In [162]:
outer_join2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109256 entries, 0 to 111080
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   publish_time   109256 non-null  object 
 1   view_counts    109256 non-null  object 
 2   summarization  109256 non-null  object 
 3   相关性            109256 non-null  object 
 4   涨跌             109256 non-null  float64
 5   x              109256 non-null  object 
dtypes: float64(1), object(5)
memory usage: 5.8+ MB


In [164]:
data_all = pd.concat([outer_join1, outer_join2], axis=0)
data_all = data_all.loc[data_all['相关性'] != 0]
data_all = data_all.reset_index(drop=True)

In [166]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111291 entries, 0 to 111290
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   publish_time   111291 non-null  object 
 1   view_counts    111291 non-null  object 
 2   summarization  111291 non-null  object 
 3   相关性            111291 non-null  object 
 4   涨跌             111291 non-null  float64
 5   x              111291 non-null  object 
dtypes: float64(1), object(5)
memory usage: 5.1+ MB


In [168]:
data_all.to_csv('./朴素贝叶斯处理过的.csv',index=False)